The Twitter REST APIs provide read and write access to existing Twitter data. There are a lot of useful APIs in this collection, but we only need to discuss the Search API for our purposes.The Twitter Search API is used to search a random sample of the past week of tweets. Note that the Search API does not capture every tweet in the past week — it aims for “relevance, not completeness.”

This will work well for pupose of capturing a representative samples of World Cup sentiment tweets from within the U.S for the past week of play (the semi-final games on Tue. 7/10 and Wed. 7/11., third place Sat.7/14 game and final Sun. 7/15). This harvest was collected on Sun. 7/15 following the conclusion of the Final (France beat Croatia). 

In [23]:
import sys
import os
import tweepy
import jsonpickle
import json

In [24]:
consumer_key="WJplbIJsJOW4l1iWvpIJA4s0K"
consumer_secret="HbyyxE5hJcUR58419edGM1SJCp07hcAzmIP3YU4DitAEJATGIj"
access_key="99735645-gRIyDayEgb8tJhPwVPk0VBGxTtNzTR3eyZH4xwZPn"
access_secret="wBC6E4bf5Yro3Ycyst93QS7cuNJq9CtkOpdBG2e8WRi6W"

In [25]:
#Pass our consumer key and consumer secret to Tweepy's user authentication handler
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

#Pass our access token and access secret to Tweepy's user authentication handler
auth.set_access_token(access_key, access_secret)

#Creating a twitter API wrapper using tweepy
api = tweepy.API(auth)

#Error handling
if (not api):
    print ("Problem connecting to API")
    
#Getting Geo ID for U.S
places = api.geo_search(query="USA", granularity="country")

#Copy USA id
place_id = places[0].id
print('USA id is: ',place_id)

USA id is:  96683cc9126741d1


In [26]:
#Switching to application authentication
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

#Setting up new api wrapper, using authentication only
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

#Error handling
if (not api):
    print ("Problem Connecting to API")

#You can check how many queries you have left using rate_limit_status() method
api.rate_limit_status()['resources']['search']

#This is what we are searching for
#We can restrict the location of tweets using place:id 
#We can search for multiple phrases using OR
searchQuery = 'place:96683cc9126741d1 #France OR #Croatia OR #England OR #Belgium OR #WorldCup OR #WorldCupFinal'
        

In [27]:
#Maximum number of tweets we want to collect 
maxTweets = 1000000

#The twitter Search API allows up to 100 tweets per query
tweetsPerQry = 100

tweetCount = 0

#Open a text file to save the tweets to
with open('worldcupsentiments07172018.json', 'w') as f:

    #Tell the Cursor method that we want to use the Search API (api.search)
    #Also tell Cursor our query, and the maximum number of tweets to return
    for tweet in tweepy.Cursor(api.search,q=searchQuery).items(maxTweets) :         

        #Verify the tweet has place info before writing (It should, if it got past our place filter)
        if tweet.place is not None:
            
            #Write the JSON format to the text file, and add one to the number of tweets we've collected
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
            tweetCount += 1

    #Display how many tweets we have collected
    print("Downloaded {0} tweets".format(tweetCount))

Downloaded 33 tweets


In [28]:
#Let's look at what we are actually printing to text
tweet._json

{'contributors': None,
 'coordinates': {'coordinates': [-115.16716007, 36.12705064], 'type': 'Point'},
 'created_at': 'Tue Jul 17 01:41:47 +0000 2018',
 'entities': {'hashtags': [{'indices': [54, 73], 'text': 'sportshandicapping'},
   {'indices': [74, 86], 'text': 'handicapper'},
   {'indices': [87, 100], 'text': 'handicapping'},
   {'indices': [101, 105], 'text': 'mlb'},
   {'indices': [106, 115], 'text': 'baseball'}],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/1…',
    'expanded_url': 'https://twitter.com/i/web/status/1019034395531993088',
    'indices': [117, 140],
    'url': 'https://t.co/MvUHQoTP9o'}],
  'user_mentions': []},
 'favorite_count': 1,
 'favorited': False,
 'geo': {'coordinates': [36.12705064, -115.16716007], 'type': 'Point'},
 'id': 1019034395531993088,
 'id_str': '1019034395531993088',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str

In [29]:
#A function to clean up and print all of the JSON attributes 
def PrintMembers(obj):
    for attribute in dir(obj):
        
        #We don't want to show built in methods of the class
        if not attribute.startswith('__'):
            print(attribute)
            
PrintMembers(tweet)

_api
_json
author
contributors
coordinates
created_at
destroy
entities
favorite
favorite_count
favorited
geo
id
id_str
in_reply_to_screen_name
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
is_quote_status
lang
metadata
parse
parse_list
place
possibly_sensitive
retweet
retweet_count
retweeted
retweets
source
source_url
text
truncated
user


In [30]:
import pandas as pd

In [31]:
def pop_tweets(path):
    #Declare a new data frame with pandas, with some specific column names
    tweets = pd.DataFrame(columns=['screenName','userId','text','location'])

    #Open the text file that contains the tweets we collected
    tweets_file = open(path, "r")
    
    #Read the text file line by line
    for line in tweets_file:
        
        #Load the JSON information
        tweet = json.loads(line)
        
        #If the tweet isn't empty, add it to the data frame
        if ('text' in tweet): 
            tweets.loc[len(tweets)]=[tweet['user']['screen_name'],\
            tweet['user']['id'],tweet['text'],tweet['place']['full_name']]    

    return tweets

WorldCup_tweets = pop_tweets('worldcupsentiments07172018.json')

In [32]:
WorldCup_tweets.head(n=5)

,screenName,userId,text,location
0,HABBENINK,30084883,Chapeau de poisson. #sketchbook #drawing #illu...,"Holladay, UT"
1,minda33,18010616,It is not too late for this important #WorldCu...,"Iowa, USA"
2,ronnieisrunning,364544742,"https://t.co/2JKhsekx3F 6,948 days in a row. #...","Dallas, TX"
3,johnnykera,179530315,"Like I said, we still celebrate after a loss. ...","Los Angeles, CA"
4,TexSR_Tanner1,2477561683,@cosmicfunpalace @ohJuliatweets @RobertB_Rice ...,"Flower Mound, TX"


In [33]:
print(WorldCup_tweets['text'][3])

Like I said, we still celebrate after a loss. #california #croatia #love #losangeles #Instagram #video #croat… https://t.co/Fzj4Mxu2gT
